In [ ]:
!pip install yfinance

In [34]:
import pandas as pd
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

def display_markdown(content):
    display(Markdown(content))

def get_btc_data(start_date, end_date):
    btc_data = yf.download('BTC-USD', start=start_date, end=end_date)
    return btc_data

def scrape_live_gold_data():
    gold_url = 'https://www.investing.com/commodities/gold'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(gold_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        price_element = soup.find('span', {'id': 'last_last'})
        price = float(price_element.text.replace(',', '')) 

        timestamp = datetime.now()

        return pd.DataFrame({'Timestamp': [timestamp], 'price': [price]})
    else:
        raise ValueError(f"Failed to retrieve gold data. Status code: {response.status_code}")

def preprocess_data(data):
    return data

def predict_next_price(data):
    model = LinearRegression()
    X = data.index.values.reshape(-1, 1)
    y = data['Close'] if 'Close' in data.columns else data['price']  # Assuming 'Close' column for BTC and 'price' column for gold
    model.fit(X, y)
    next_timestamp = data.index.max() + pd.Timedelta(days=1)
    next_price = model.predict([[len(data)]])[0]
    return next_timestamp, next_price

# asset_type = input("Enter asset type (gold or btc): ")
asset_type = "btc"
start_date_input = input("Enter start date (YYYY-MM-DD): ")
start_date = datetime.strptime(start_date_input, '%Y-%m-%d')

end_date_input = input("Enter end date (YYYY-MM-DD): ")
end_date = datetime.strptime(end_date_input, '%Y-%m-%d')

if asset_type.lower() == 'btc':
    asset_data = get_btc_data(start_date, end_date)
elif asset_type.lower() == 'gold':
    asset_data = scrape_live_gold_data()
else:
    raise ValueError("Invalid asset type. Please enter 'gold' or 'btc'.")

display_markdown('### Time Series')
display_markdown('### Asset Price Prediction')

display_markdown('This program predicts the next price of a specified asset (gold or BTC) using linear regression.')

display_markdown('#### Preprocessing Output:')
preprocessed_data = preprocess_data(asset_data)
print(preprocessed_data.head())

display_markdown('#### Prediction Output:')
next_timestamp, next_price = predict_next_price(asset_data)
print(f'Next Timestamp: {next_timestamp}, Predicted Next Price: ${next_price:.2f}')


Enter start date (YYYY-MM-DD):  2020-12-23
Enter end date (YYYY-MM-DD):  2023-11-23


[*********************100%%**********************]  1 of 1 completed


### Time Series

### Asset Price Prediction

This program predicts the next price of a specified asset (gold or BTC) using linear regression.

#### Preprocessing Output:

                    Open          High           Low         Close   
Date                                                                 
2020-12-23  23781.974609  24024.490234  22802.646484  23241.345703  \
2020-12-24  23240.203125  23768.337891  22777.597656  23735.949219   
2020-12-25  23733.570312  24710.101562  23463.673828  24664.791016   
2020-12-26  24677.015625  26718.070312  24522.689453  26437.037109   
2020-12-27  26439.373047  28288.839844  25922.769531  26272.294922   

               Adj Close       Volume  
Date                                   
2020-12-23  23241.345703  51146161904  
2020-12-24  23735.949219  41080759713  
2020-12-25  24664.791016  42068395846  
2020-12-26  26437.037109  48332647295  
2020-12-27  26272.294922  66479895605  


#### Prediction Output:

Next Timestamp: 2023-11-23 00:00:00, Predicted Next Price: $536972.20


In [30]:
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.arima.model import ARIMA
from IPython.display import Markdown, display

def display_markdown(content):
    display(Markdown(content))

def get_btc_data(start_date, end_date):
    btc_data = yf.download('BTC-USD', start=start_date, end=end_date)
    return btc_data['Close']

def scrape_live_gold_data():
    gold_url = 'https://www.investing.com/commodities/gold'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(gold_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        price_element = soup.find('span', {'id': 'last_last'})
        price = float(price_element.text.replace(',', ''))  

        timestamp = datetime.now()

        return pd.DataFrame({'Timestamp': [timestamp], 'price': [price]})
    else:
        raise ValueError(f"Failed to retrieve gold data. Status code: {response.status_code}")

def preprocess_data(data):
    return data

def predict_next_prices(data, steps=3):
    model = ARIMA(data, order=(5,1,0)) 
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=steps)
    
    next_timestamps = [data.index[-1] + timedelta(days=i) for i in range(1, steps + 1)]
    next_prices = forecast.values

    return pd.DataFrame({'Timestamp': next_timestamps, 'Predicted Price': next_prices})

asset_type = "btc"
start_date_input = input("Enter start date (YYYY-MM-DD): ")
start_date = datetime.strptime(start_date_input, '%Y-%m-%d')

end_date_input = input("Enter end date (YYYY-MM-DD): ")
end_date = datetime.strptime(end_date_input, '%Y-%m-%d')

if asset_type.lower() == 'btc':
    asset_data = get_btc_data(start_date, end_date)
    prediction_output = predict_next_prices(asset_data)
elif asset_type.lower() == 'gold':
    asset_data = scrape_live_gold_data()
    prediction_output = preprocess_data(asset_data)  
else:
    raise ValueError("Invalid asset type. Please enter 'gold' or 'btc'.")

display_markdown('## Time Series')
display_markdown('### BTC Price Prediction')

display_markdown('This program predicts the next prices of a specified asset (gold or BTC) using the ARIMA model.')

display_markdown('#### Preprocessing Output:')
print(preprocess_data(asset_data))

display_markdown('#### Prediction Output:')
print(prediction_output)


Enter start date (YYYY-MM-DD):  2001-12-12
Enter end date (YYYY-MM-DD):  2023-11-23


[*********************100%%**********************]  1 of 1 completed


C:\Users\aliga\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\aliga\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\aliga\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


## Time Series

### BTC Price Prediction

This program predicts the next prices of a specified asset (gold or BTC) using the ARIMA model.

#### Preprocessing Output:

Date
2014-09-17      457.334015
2014-09-18      424.440002
2014-09-19      394.795990
2014-09-20      408.903992
2014-09-21      398.821014
                  ...     
2023-11-18    36585.703125
2023-11-19    37386.546875
2023-11-20    37476.957031
2023-11-21    35813.812500
2023-11-22    37432.339844
Name: Close, Length: 3354, dtype: float64


#### Prediction Output:

   Timestamp  Predicted Price
0 2023-11-23     37399.825714
1 2023-11-24     37394.532829
2 2023-11-25     37378.317561
